In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.webdriver.chrome.options import Options
import pandas as pd
import random
import math

# ヘッドレス、シークレットモード、ユーザーエージェントをランダムで変更
options = Options()
options.add_argument('incognito')
options.add_experimental_option("excludeSwitches", ['enable-automation'])
options.add_argument('--headless')

user_agent = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.2 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36']

ua = user_agent[random.randrange(0, len(user_agent), 1)]
options.add_argument('--user-agent=' + ua)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 暗黙的待機　要素を取得するまで最大引数秒分待機してね。
# driver.implicitly_wait(10)

# 最大の読み込み時間を設定 今回は最大30秒待機できるようにする
wait_ = WebDriverWait(driver=driver, timeout=30)
# 要素が全て検出できるまで待機する
wait = wait_.until(EC.presence_of_all_elements_located)

url = f'https://crowdworks.jp/login'
driver.get(url)

wait
userName = driver.find_element(by=By.CSS_SELECTOR, value='#username')
userName.click()
userName.send_keys('wataruwatanabe0408@gmail.com')
password = driver.find_element(by=By.CSS_SELECTOR, value='#password')
password.click()
password.send_keys('kaGpib-8pimhu-mubkyt')
driver.find_element(by=By.CSS_SELECTOR, value='#vue-container > div > main > div > div > div > div > div.stack.stack-24 > div.container > div > div.stack.stack-24 > div.stack.stack-16 > div.stack.stack-24 > div > form > div > div:nth-child(2) > button').click()

wait
url = f'https://crowdworks.jp/r/job_offers/9162860/proposals?filter=proposed'
driver.get(url)

wait
_lists = driver.find_elements(by=By.LINK_TEXT, value='詳細へ')

data = []
lists = []
for i in _lists:
    lists.append(i.get_attribute('href'))
    
for list in lists:
    wait
    driver.get(list)
    
    wait
    name = driver.find_element(by=By.CSS_SELECTOR, value='#Content > div.cw-global_row > div.progress > div.progress_detail > div > table > tbody > tr > td.user > a').text
    _price = driver.find_element(by=By.CSS_SELECTOR, value='#Content > div.cw-global_row > div.progress > div.progress_detail > div > table > tbody > tr > td.condition').text
    target = '\n'
    idx = _price.find(target)
    price: int = _price[:idx].replace('固定報酬: ','').replace('円','')
    date: date = driver.find_element(by=By.CSS_SELECTOR, value='#Content > div.cw-global_row > div.progress > div.progress_detail > div > table > tbody > tr > td.created_at').text
    detail = driver.find_element(by=By.CSS_SELECTOR, value='#pack-message-thread > div > div.intro-employer_talking.intro-employer_proposed_project > div > div:nth-child(2) > div.cw-media_body > div.body > p').text

    files = []
    _files = driver.find_elements(by=By.CSS_SELECTOR, value='#pack-message-thread > div > div.intro-employer_talking.intro-employer_proposed_project > div > div.cw-media_foot > div > div:nth-child(2) > a')

    if _files == []:
        print('要素は何も有りませんでした。')
    else:
        for i in _files:
            files.append(i.get_attribute('href'))

    datum ={
        '日付':date,
        '名前':name,
        '金額':price,
        '詳細':detail,
        '添付ファイル':files
    }
    
    data.append(datum)
    print(f'{name}追加')

data_summary = pd.DataFrame(data)
data_summary[['添付ファイル1', '添付ファイル2']] = pd.DataFrame(data_summary['添付ファイル'].tolist(), index=data_summary.index)
data_summary.drop('添付ファイル', axis=1, inplace=True)
data_summary.to_csv('info.csv', encoding='utf_8_sig', index=False)

In [ ]:
driver.quit()